In [1]:
import midas
import json
from pathlib import Path

In [2]:
from midas.db.migrate import get_alembic_config

In [3]:
from sqlalchemy import create_engine
import alembic.command as cmds

In [4]:
from tqdm import tqdm

# Files

In [5]:
db1 = '/home/jared/projects/midas/data/reference-db-v2/midas-reference-v2.db'
db2 = '/home/jared/projects/midas/data/reference-db-v2/midas-reference-v2-d961d0698083.db'

In [6]:
! cp {db1} {db2}

# Connect

In [7]:
engine1 = create_engine('sqlite:///' + db1)
engine2 = create_engine('sqlite:///' + db2)

# Migrate

This takes a while...

In [8]:
aconf = get_alembic_config(engine2)

In [9]:
cmds.upgrade(aconf, 'd961d0698083')

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 4815cccfb01b -> d961d0698083, Remove unneeded columns


# Check

In [10]:
import sqlalchemy as sa
from sqlalchemy import sql
from midas.db import sqla

In [11]:
genomes = sql.table(
    'genomes',
    sa.Column('id', sa.Integer(), primary_key=True),
    sa.Column('is_assembled', sa.Boolean()),
    sa.Column('ncbi_taxid', sa.Integer(), index=True),
    sa.Column('entrez_summary', sqla.JsonType),
    sa.Column('extra', sqla.JsonType),
)

reference_genome_sets = sql.table(
    'reference_genome_sets',
    sa.Column('id', sa.Integer(), primary_key=True),
    sa.Column('signatureset_key', sa.String()),
    sa.Column('signatureset_version', sa.String()),
    sa.Column('extra', sqla.JsonType),
)

taxa = sql.table(
    'taxa',
    sa.Column('id', sa.Integer(), primary_key=True),
    sa.Column('entrez_data', sqla.JsonType),
    sa.Column('extra', sqla.JsonType),
)

In [12]:
conn1 = engine1.connect()
conn2 = engine2.connect()

## Validate genomes

In [13]:
for row in tqdm(conn1.execute(genomes.select())):
    extra1 = row.extra.as_builtin()
    
    extra2, = conn2.execute(sql.select([genomes.c.extra]).where(genomes.c.id==row.id)).fetchone()
    extra2 = extra2.as_builtin()
    
    assert extra2['is_assembled'] == row.is_assembled
    assert extra2['ncbi_taxid'] == row.ncbi_taxid
    assert extra2['entrez_summary'] == row.entrez_summary.as_builtin()
    
    for k, v in extra1.items():
        assert extra2[k] == v

74160it [00:35, 2094.88it/s]


## Validate taxa

In [15]:
for row in tqdm(conn1.execute(taxa.select())):
    extra1 = row.extra.as_builtin()
    
    extra2, = conn2.execute(sql.select([taxa.c.extra]).where(taxa.c.id==row.id)).fetchone()
    extra2 = extra2.as_builtin()
    
    assert extra2.get('entrez_data') == (None if row.entrez_data is None else row.entrez_data.as_builtin())
    
    for k, v in extra1.items():
        assert extra2[k] == v

2424it [00:00, 3158.01it/s]
